# Add Traffic Signals to Network
In this case we already have a traffic signal inventory from the Georgia Department of Transportation, but code for downloading existing traffic signal data from OpenStreetMap (OSM) is also included. This code also retrieves crossings from OSM.

First, we'll find signals in the GDOT data that are not covered in the OSM data. Then we'll take these intersect the them with OSM nodes. Using the reference ID columns on the OSM links, these signals can be added to the links. A turn dataframe is then constructed with all of the links that contain at least one signal.

Once this happens, the link street name will be cross-referenced to validate the match.

- If signals are both road links, check the road name on each to see if it matches the GDOT name
- If non-road link just set it to null for now. Some of these may be crosswalks at the intersection, but they could also be walkways further away from the intersection.

This final result should be QAQC'd.

In [1]:
from pathlib import Path 
import geopandas as gpd
import pandas as pd

from bikewaysim.paths import config
from bikewaysim.network import conflation_tools, modeling_turns

In [2]:
links = gpd.read_file(config['network_fp']/'networks.gpkg',layer='osm_links')
nodes = gpd.read_file(config['network_fp']/'networks.gpkg',layer='osm_nodes')
#add attributes back
raw = gpd.read_file(config['network_fp']/f"osm.gpkg",layer="raw",ignore_geometry=True)
links = pd.merge(links,raw[['osmid','highway','name']],how='left',on='osmid')
del raw
#create a name col for checking against the GDOT names
links['name0'] = links['name'].apply(lambda x: conflation_tools.contract_suffix(x))

In [3]:
#buffer function
def buffer_signals(signal_gdf,buffer_ft):
    '''
    Use to create a copy of a gdf and buffer the point geometry
    '''
    signal_gdf = signal_gdf.copy()
    signal_gdf.geometry = signal_gdf.buffer(buffer_ft)
    return signal_gdf

# OSM Signals
No need to conflate, already embedded in the OSM network.

In [4]:
osm_signals = gpd.read_file(config['network_fp']/f"osm.gpkg",layer='highway_nodes')
osm_signals = osm_signals[osm_signals['highway']=='traffic_signals']
osm_signals.to_crs(config['projected_crs_epsg'],inplace=True)
osm_signal_ids = set(osm_signals['osmid'].tolist())

In [5]:
signalized_links = links[(links['A'].isin(osm_signal_ids)) | (links['B'].isin(osm_signal_ids))]

## Create turn graph dataframe
_, turns_df = modeling_turns.create_pseudo_dual_graph(signalized_links,'A','B','linkid','oneway')

#add signals ids back in
turns_df['signalized'] = turns_df['source_B'].isin(osm_signal_ids)

#drop unsignalized turn movements
turns_df = turns_df[turns_df['signalized']==True]

#export
turns_df.to_csv(config['network_fp']/'osm_signals.csv',index=False)

# GDOT Signals

In [6]:
# read gdot signals
keep = ['signalID','mainStreetName','sideStreetName','geometry']
gdot_signals = gpd.read_file(config['gdot_signals_fp'],mask=gpd.read_file(config['studyarea_fp'])).to_crs(links.crs)[keep]
gdot_signals.head()

,signalID,mainStreetName,sideStreetName,geometry
0,1,Luckie St NW,Merritts Ave NW,POINT (5398770.817 780136.625)
1,12,Ivan Allen Jr Blvd,Williams St/I-75 Ramp/I-85 Ramp,POINT (5399447.406 779919.22)
2,12,Ivan Allen Jr Blvd,Williams St/I-75 Ramp/I-85 Ramp,POINT (5399447.406 779919.22)
3,14,West Peachtree St NW,Ivan Allen Jr Blvd NE,POINT (5399685.52 780043.271)
4,15,Ralph McGill Blvd NE,Peachtree St NE,POINT (5399845.355 780081.852)


In [7]:
import re
gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName']
gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName'] 
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'\(.*?\)', '', x)) # remove anything in parenthesis #TODO instead put / / instead so it can be sperated out
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'\(.*?\)', '', x)) # remove anything in parenthesis
gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: x.replace('MLK','martin luther king'))#re.sub(r'MLK', 'martin luther king', x)) # change mlk to full version
gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: x.replace('MLK','martin luther king'))#re.sub(r'MLK', 'martin luther king', x)) # change mlk to full version
gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: x.replace('RDA','ralph david abernathy'))#re.sub(r'RDA', 'ralph david abernathy', x)) # change mlk to full version
gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: x.replace('RDA','ralph david abernathy'))# re.sub(r'RDA', 'ralph david abernathy', x)) # change mlk to full version
gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'[()]', '/', x)) # remove anything in parenthesis
gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'[()]', '/', x)) # remove anything in parenthesis
mainStreetName = gdot_signals['mainStreetName0'].apply(lambda x: x.split('/'))
sideStreetName = gdot_signals['sideStreetName0'].apply(lambda x: x.split('/'))
gdot_signals['gdot_names'] = mainStreetName + sideStreetName
gdot_signals['gdot_names'] = gdot_signals['gdot_names'].apply(lambda x: [y for y in x if len(y.replace(' ','')) > 0])
gdot_signals['gdot_names'] = gdot_signals['gdot_names'].apply(lambda list_of_names: [conflation_tools.contract_suffix(name) for name in list_of_names])
gdot_signals.sample(20)

# #some specific regex for the gdot dataset
# import re 

# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: x.lower())
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'sr \d+', '', x)) # remove the state routes
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'us \d+', '', x)) # remove the us routes
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'ga \d+', '', x)) # remove the ga routes
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'i-\d+', '', x)) # remove the us routes
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'phb \d+', '', x)) # remove the us routes
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub(r'\(.*?\)', '', x)) # remove anything in parenthesis
# # gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub('/', ' ', x)) # replace / with space
# gdot_signals['mainStreetName0'] = gdot_signals['mainStreetName0'].apply(lambda x: re.sub('ramp', ' ', x)) # remove ramps from name

# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: x.lower())
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'sr \d+', '', x)) # remove the state routes
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'us \d+', '', x)) # remove the us routes
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'ga \d+', '', x)) # remove the ga routes
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'i-\d+', '', x)) # remove the us routes
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'phb \d+', '', x)) # remove the us routes
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub(r'\(.*?\)', '', x)) # remove anything in parenthesis
# # gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub('/', ' ', x)) # replace / with space
# gdot_signals['sideStreetName0'] = gdot_signals['sideStreetName0'].apply(lambda x: re.sub('ramp', ' ', x)) # remove ramps from name

,signalID,mainStreetName,sideStreetName,geometry,mainStreetName0,sideStreetName0,gdot_names
223,216,Juniper St,11th St,POINT (5399307.084 782093.414),Juniper St,11th St,"[juniper st, 11th st]"
26,47,Andrew Young International Blvd NE,Piedmont Ave NE,POINT (5400368.452 779716.787),Andrew Young International Blvd NE,Piedmont Ave NE,"[andrew young international blvd ne, piedmont ..."
273,56,Andrew Young International Blvd,Williams St NW,POINT (5399652.923 779407.346),Andrew Young International Blvd,Williams St NW,"[andrew young international blvd, williams st nw]"
84,256,10th Street NE,Piedmont Park Pedestrian Crossing/The Children...,POINT (5399858.616 782183.674),10th Street NE,Piedmont Park Pedestrian Crossing/The Children...,"[10th st ne, piedmont park pedestrian crossing..."
189,8611,17th St,Mecaslin St,POINT (5397351.244 782175.911),17th St,Mecaslin St,"[17th st, mecaslin st]"
62,217,10th St NE,Juniper St,POINT (5399352.95 781956.641),10th St NE,Juniper St,"[10th st ne, juniper st]"
108,7015,SR 42 / Moreland Avenue,Commercial Drive,POINT (5403304.635 780675.217),SR 42 / Moreland Avenue,Commercial Drive,"[sr 42, moreland ave, commercial dr]"
258,472,10th St NE,Myrtle St NE,POINT (5399613.851 782074.914),10th St NE,Myrtle St NE,"[10th st ne, myrtle st ne]"
67,227,Piedmont Ave NE,Currier St NE,POINT (5400115.265 780320.854),Piedmont Ave NE,Currier St NE,"[piedmont ave ne, currier st ne]"
33,64,Luckie St,Cone St,POINT (5399772.766 779238.588),Luckie St,Cone St,"[luckie st, cone st]"


## Symmetric Difference
Find GDOT signals that are not already covered by OSM. Maybe ask an undergrad to add these into OSM later.

In [8]:
print(gdot_signals.shape[0],'GDOT signals and',osm_signals.shape[0],'OSM signals')
buffer_ft = 100 #selecting 100 ft based on city block sizes and that's about where the number of gdot signals not represented by osm signals drops
buffered_osm_signals = buffer_signals(osm_signals,buffer_ft)
difference = gdot_signals.overlay(buffered_osm_signals,how='difference')
difference = difference[difference.drop(columns=['gdot_names']).duplicated()==False]
print('Around',difference.shape[0],'GDOT traffic signals not in OSM')

334 GDOT signals and 318 OSM signals
Around 68 GDOT traffic signals not in OSM


## Add these new GDOT signals into the OSM network
Set a buffer for each signal and find all candidate nodes associated
- Filter OSM nodes to only consider links with a node with a degree higher than 2 and that's labelled as a road
- Buffer and intersect the GDOT signals with the filtered OSM nodes
- Check the road names for links attached to a candidate node


we set a buffer distance around each signal to find all candidate nodes associated with the traffic signal.
- Then, we remove links that are unlikely to be signalized intersections

In [9]:
from collections import Counter

#only consider road nodes (if there's a service road that connects to a signalized intersection this should still count them)
only_roads = links[links['link_type'].isin(['road'])].copy()
road_nodes = pd.Series(Counter(only_roads['A'].tolist()+only_roads['B'].tolist()))
road_nodes = set(road_nodes[road_nodes>2].index.tolist()) # and remove matches where degree is 2 or less
road_nodes = nodes[nodes['N'].isin(road_nodes)]

buffer_ft = 100
gdot_buffered = buffer_signals(difference,buffer_ft)
candidate_signals = gpd.overlay(road_nodes,gdot_buffered,how="intersection")
# candidate_signals.explore()

# subset the links
candidate_signals0 = set(candidate_signals['N'].tolist())
candidate_links = links[links['A'].isin(candidate_signals0) | links['B'].isin(candidate_signals0)].copy()
candidate_links = candidate_links[candidate_links['link_type']=='road']

In [10]:
# add the candidate nodes info to the candidate links so that we can check street name
from importlib import reload
reload(conflation_tools)
A = candidate_links.merge(candidate_signals,left_on='A',right_on='N')
name_check = A.apply(lambda row: any([conflation_tools.name_check(row['name0'],name,.6) for name in row['gdot_names']]), axis = 1)
# A[name_check==False].sample(30)
A = A[name_check]
A = set(A['linkid'].tolist())

B = candidate_links.merge(candidate_signals,left_on='B',right_on='N')
name_check = B.apply(lambda row: any([conflation_tools.name_check(row['name0'],name,.6) for name in row['gdot_names']]), axis = 1)
# main_check = B.apply(lambda row: conflation_tools.name_check(row['name'],row['mainStreetName']),axis=1)
# side_check = B.apply(lambda row: conflation_tools.name_check(row['name'],row['sideStreetName']),axis=1)
B = B[name_check]
B = set(B['linkid'].tolist())

candidate_links = links[links['linkid'].isin(set.union(A,B))]
# candidate_links.explore()

In [11]:
_, turns_df = modeling_turns.create_pseudo_dual_graph(candidate_links,'osm_A','osm_B','osm_linkid','oneway')

In [12]:
#add signals ids back in
turns_df['signalized'] = turns_df['source_B'].isin(candidate_signals0)
turns_df = turns_df[turns_df['signalized'] == True]

# turns_df.drop(columns=['source','target'],inplace=True)

turns_df.to_csv(config['network_fp']/'gdot_signals.csv')